# The Attention mechanism and its implementation in Keras

Computing self-attention of a sentence with GloVe embeddings and the `MultiHeadAttention` class in Keras

Documents used to write the notebook:
* Chollet's book, _Deep Learning with Python_, Second Edition, 2021, pp. 339-342
* The Keras documentation: https://keras.io/api/layers/attention_layers/multi_head_attention/#multiheadattention-layer
* Tensorflow documentation: https://www.tensorflow.org/api_docs/python/tf/keras/layers/MultiHeadAttention
* Implementation: https://github.com/keras-team/keras/blob/v2.7.0/keras/layers/multi_head_attention.py

Author: Pierre Nugues

## Modules

In [1]:
import numpy as np
import numpy.linalg
from scipy.special import softmax
import tensorflow as tf

## Noncontextual embeddings

We load GloVe

In [2]:
def load(file):
    """
    Return the embeddings in the from of a dictionary
    :param file:
    :return:
    """
    file = file
    embeddings = {}
    glove = open(file)
    for line in glove:
        values = line.strip().split()
        word = values[0]
        vector = np.array(values[1:], dtype='float32')
        embeddings[word] = vector
    glove.close()
    embeddings_dict = embeddings
    embedded_words = sorted(list(embeddings_dict.keys()))
    return embeddings_dict

In [3]:
embedding_file = '/Users/pierre/Documents/Cours/EDAN20/corpus/glove.6B.50d.txt'
embeddings_dict = load(embedding_file)

In [4]:
embeddings_dict['ship']

array([ 1.5213  ,  0.10522 ,  0.38162 , -0.50801 ,  0.032423, -0.13484 ,
       -1.2474  ,  0.79813 ,  0.84691 , -1.101   ,  0.88743 ,  1.3749  ,
        0.42928 ,  0.65717 , -0.2636  , -0.41759 , -0.48846 ,  0.91061 ,
       -1.7158  , -0.438   ,  0.78395 ,  0.19636 , -0.40657 , -0.53971 ,
        0.82442 , -1.7434  ,  0.14285 ,  0.28037 ,  1.1688  ,  0.16897 ,
        2.2271  , -0.58273 , -0.45723 ,  0.62814 ,  0.54441 ,  0.28462 ,
        0.44485 , -0.55343 , -0.36493 , -0.016425,  0.40876 , -0.87148 ,
        1.5513  , -0.80704 , -0.10036 , -0.28461 , -0.33216 , -0.50609 ,
        0.48272 , -0.66198 ], dtype=float32)

## Cosine similarity

Let us compute the cosine similarity of word in a sentence:
> I must go back to my ship and to my crew

_Odyssey_, book I 

In [5]:
sentence = 'I must go back to my ship and to my crew'

In [6]:
words = sentence.lower().split()
words

['i', 'must', 'go', 'back', 'to', 'my', 'ship', 'and', 'to', 'my', 'crew']

We build the embedding matrix

In [7]:
embeddings_seq = []
for word in words:
    embeddings_seq += [embeddings_dict[word]]
embeddings_seq = np.array(embeddings_seq)
#embeddings_seq

We compute the attention scores: pairwise cosine of the words

In [8]:
attn_scores = np.zeros((len(words),len(words)))
for i in range(len(words)):
    scores = np.zeros(len(words))
    for j in range(len(words)):
        scores[j] = (np.dot(embeddings_seq[i], embeddings_seq[j])/
                     (np.linalg.norm(embeddings_seq[i]) * 
                      np.linalg.norm(embeddings_seq[j])))
        #scores[j] = np.dot(embeddings_dict[words[i]], embeddings_dict[words[j]])
    attn_scores[i] = scores

In [9]:
print('\t', end='')
for i in range(len(words)):
    print(words[i], end='\t')
print()

for i in range(attn_scores.shape[0]):
    print(words[i], end='\t')
    for j in range(attn_scores.shape[1]):
        print(f"{attn_scores[i,j]:.2f}", end='\t')
    print()

	i	must	go	back	to	my	ship	and	to	my	crew	
i	1.00	0.75	0.86	0.76	0.73	0.90	0.35	0.65	0.73	0.90	0.42	
must	0.75	1.00	0.85	0.68	0.87	0.69	0.42	0.69	0.87	0.69	0.45	
go	0.86	0.85	1.00	0.84	0.84	0.81	0.41	0.68	0.84	0.81	0.49	
back	0.76	0.68	0.84	1.00	0.83	0.76	0.49	0.77	0.83	0.76	0.51	
to	0.73	0.87	0.84	0.83	1.00	0.68	0.54	0.86	1.00	0.68	0.51	
my	0.90	0.69	0.81	0.76	0.68	1.00	0.38	0.63	0.68	1.00	0.44	
ship	0.35	0.42	0.41	0.49	0.54	0.38	1.00	0.46	0.54	0.38	0.78	
and	0.65	0.69	0.68	0.77	0.86	0.63	0.46	1.00	0.86	0.63	0.49	
to	0.73	0.87	0.84	0.83	1.00	0.68	0.54	0.86	1.00	0.68	0.51	
my	0.90	0.69	0.81	0.76	0.68	1.00	0.38	0.63	0.68	1.00	0.44	
crew	0.42	0.45	0.49	0.51	0.51	0.44	0.78	0.49	0.51	0.44	1.00	


## Contextual embeddings

We design a new embedding representation for _ship_ so that it receives an influence from _crew_ and the other words of its context. This influence will depend on the noncontextual embeddings. The attention scores are the cosine similarities

In [10]:
attn_scores[6]

array([0.34663907, 0.41782767, 0.40681112, 0.48531651, 0.54014385,
       0.3791028 , 1.        , 0.45863339, 0.54014385, 0.3791028 ,
       0.78480232])

We compute the new embeddings as the sum of the noncontextual embeddings weighted by the cosine similarity. We have contextual embeddings.

In [11]:
new_embeddings_ship = (0.35 * embeddings_dict['i'] + 
                  0.42 * embeddings_dict['must'] + 
                  0.41 * embeddings_dict['go'] +
                  0.49* embeddings_dict['back'] +
                  0.54* embeddings_dict['to'] + 
                  0.38* embeddings_dict['my'] +
                  1.00* embeddings_dict['ship'] +
                  0.46* embeddings_dict['and'] +
                  0.54* embeddings_dict['to'] +
                  0.38* embeddings_dict['my'] +
                  0.78* embeddings_dict['crew'])
new_embeddings_ship

array([  3.2289004 ,   0.6421813 ,   1.4712307 ,  -2.3537598 ,
         2.24136   ,  -0.42374972,  -4.105233  ,   2.6215937 ,
         0.17187847,  -2.4323788 ,   1.3882339 ,   3.7241364 ,
        -1.9721073 ,   1.1893367 ,   2.2511206 ,   0.9501926 ,
        -0.76461965,   1.0288985 ,  -3.0553396 ,  -3.6306143 ,
         0.8304751 ,   2.9298651 ,   1.3221488 ,  -0.70915157,
         2.9745216 , -10.595905  ,  -1.3167882 ,   0.20589754,
         3.5456927 ,  -2.7711318 ,  18.2672    ,   2.4816926 ,
        -3.588689  ,   0.32967418,   1.2717707 ,   0.653944  ,
         1.5873263 ,   0.01946718,   0.7724056 ,  -1.4620132 ,
        -0.2066631 ,  -1.2463707 ,   2.1504393 ,  -0.18107067,
        -0.5025929 ,  -0.2888131 ,  -0.5059958 ,  -1.9675692 ,
        -0.06049497,  -0.6725442 ], dtype=float32)

Implementation with numpy

In [12]:
sum(embeddings_seq * attn_scores[6].T[:, None])

array([ 3.23191333e+00,  6.40820291e-01,  1.47175971e+00, -2.34335986e+00,
        2.23580736e+00, -4.18774560e-01, -4.10024511e+00,  2.62113565e+00,
        1.80098590e-01, -2.43597248e+00,  1.39229628e+00,  3.71878745e+00,
       -1.96033551e+00,  1.19803266e+00,  2.23935332e+00,  9.37625410e-01,
       -7.70491710e-01,  1.03488285e+00, -3.06148983e+00, -3.62586930e+00,
        8.34011570e-01,  2.92812823e+00,  1.31648467e+00, -7.13029835e-01,
        2.96666448e+00, -1.05669432e+01, -1.30994248e+00,  2.02828896e-01,
        3.53620975e+00, -2.75710038e+00,  1.82203369e+01,  2.46983156e+00,
       -3.58043840e+00,  3.26042563e-01,  1.27600905e+00,  6.57010953e-01,
        1.58887761e+00,  1.15708439e-02,  7.66195274e-01, -1.45595292e+00,
       -2.03622785e-01, -1.24835755e+00,  2.15496249e+00, -1.87666416e-01,
       -5.02529457e-01, -2.91283600e-01, -5.10062909e-01, -1.95960099e+00,
       -5.88529337e-02, -6.73801397e-01])

## Self-attention

Vaswani et al. (2017) defined attention as:
$$
\text{Attention}(\mathbf{Q}, \mathbf{K}, \mathbf{Q}) = \text{softmax}(\frac{\mathbf{Q}  \mathbf{K}^\intercal}{\sqrt{d_k}})  \mathbf{V},
$$
where
$$
\begin{array}{lcl}
\mathbf{Q} &=& \mathbf{X} \mathbf{W}_Q,   \\
\mathbf{K} &=& \mathbf{X} \mathbf{W}_K , \\
\mathbf{V} &=& \mathbf{X} \mathbf{W}_V.\\
\end{array}
$$
and $\mathbf{X}$ represents complete input sequence (all the tokens).

$d_k$ is the dimension of the input and $\sqrt{d_k}$ a scaling factor. The $\text{softmax}$ function is defined as:
$$
\text{softmax}(x_1, x_2, ..., x_j, ..., x_n) = (\frac{e^{x_1}}{\sum_{i=1}^n e^{x_i}}, \frac{e^{x_2}}{\sum_{i=1}^n e^{x_i}}, ..., \frac{e^{x_j}}{\sum_{i=1}^n e^{x_i}}, ..., \frac{e^{x_n}}{\sum_{i=1}^n e^{x_i}})
$$

We omit the weight matrices and we use the same embeddings for $\mathbf{Q}$, $\mathbf{K}$, and $\mathbf{Q}$: GloVe embeddings

For the matrix above, self attention for _ship_ yields:

In [13]:
attn_scores_vaswani = softmax(np.dot(embeddings_seq, embeddings_seq.T)
                              /np.sqrt(embeddings_dict['i'].shape), axis=-1)[6]
attn_scores_vaswani

array([0.03030739, 0.03024587, 0.02764812, 0.0406623 , 0.04593486,
       0.03426556, 0.55297636, 0.02968701, 0.04593486, 0.03426556,
       0.12807212])

We have the weights of 55% for _ship_ and 13% for _crew_, the rest from the other words.

And the new contextual embedding is for _ship:_

In [14]:
sum(embeddings_seq * attn_scores_vaswani.T[:, None])

array([ 1.0387307 ,  0.10328993,  0.34260821, -0.43199392,  0.2236531 ,
       -0.09583739, -0.99261578,  0.66616271,  0.4423922 , -0.7941921 ,
        0.56381569,  0.99210325,  0.02053569,  0.50824176,  0.07430461,
       -0.17727756, -0.34077056,  0.56745014, -1.15453678, -0.57175906,
        0.42881261,  0.41905235, -0.06575875, -0.33385476,  0.66821521,
       -1.74733617, -0.04854005,  0.15311078,  0.86423044, -0.14474712,
        2.65712497, -0.05447188, -0.53432358,  0.31597919,  0.40407802,
        0.22768488,  0.39576729, -0.29159369, -0.11262517, -0.13846768,
        0.1743577 , -0.53750545,  0.94985317, -0.41448157, -0.10386168,
       -0.17551405, -0.22132868, -0.39945137,  0.21188122, -0.36097601])

## Chollet's implementation

Now we follow Chollet's book (2021), page 339, to outline the computation. The function below is drawn his the book, page 339, and is slightly modified.

In [15]:
def self_attention(input_sequence):
    output = np.zeros(shape=input_sequence.shape) 
    attn_scores = []
    # The output will consist of contextual embeddinsgs of the same shape
    for i, pivot_vector in enumerate(input_sequence):
        scores = np.zeros(shape=(len(input_sequence),)) 
        for j, vector in enumerate(input_sequence):
            scores[j] = np.dot(pivot_vector, vector.T) # Q K^T
        scores /= np.sqrt(input_sequence.shape[1]) # sqrt(d_k)
        scores = softmax(scores) # softmax(Q K^T / sqrt(d_k))
        attn_scores += [scores]
        new_pivot_representation = np.zeros(shape=pivot_vector.shape) 
        for j, vector in enumerate(input_sequence):
             new_pivot_representation += vector * scores[j]
        output[i] = new_pivot_representation
    return output, np.array(attn_scores)

As input sequence, we use the GloVe embeddings of the words again:

In [16]:
words

['i', 'must', 'go', 'back', 'to', 'my', 'ship', 'and', 'to', 'my', 'crew']

In [17]:
embeddings_dict['ship']

array([ 1.5213  ,  0.10522 ,  0.38162 , -0.50801 ,  0.032423, -0.13484 ,
       -1.2474  ,  0.79813 ,  0.84691 , -1.101   ,  0.88743 ,  1.3749  ,
        0.42928 ,  0.65717 , -0.2636  , -0.41759 , -0.48846 ,  0.91061 ,
       -1.7158  , -0.438   ,  0.78395 ,  0.19636 , -0.40657 , -0.53971 ,
        0.82442 , -1.7434  ,  0.14285 ,  0.28037 ,  1.1688  ,  0.16897 ,
        2.2271  , -0.58273 , -0.45723 ,  0.62814 ,  0.54441 ,  0.28462 ,
        0.44485 , -0.55343 , -0.36493 , -0.016425,  0.40876 , -0.87148 ,
        1.5513  , -0.80704 , -0.10036 , -0.28461 , -0.33216 , -0.50609 ,
        0.48272 , -0.66198 ], dtype=float32)

In [18]:
embeddings_seq.shape

(11, 50)

We compute the new embeddings and the attentions scores. The result is a pair.

In [19]:
attn_output_sequence = self_attention(embeddings_seq)

In [20]:
(len(attn_output_sequence), 
 attn_output_sequence[0].shape, 
 attn_output_sequence[1].shape)

(2, (11, 50), (11, 11))

Attention scores for _ship_

In [21]:
attn_output_sequence[1][6]

array([0.0303074 , 0.03024588, 0.02764812, 0.04066233, 0.04593488,
       0.03426557, 0.55297623, 0.02968703, 0.04593488, 0.03426557,
       0.12807209])

The new contextual embeddings for _ship:_

In [22]:
attn_output_sequence[0][6]

array([ 1.03873055,  0.10328993,  0.34260817, -0.43199395,  0.22365316,
       -0.09583739, -0.99261573,  0.66616262,  0.44239205, -0.79419196,
        0.56381559,  0.99210317,  0.02053557,  0.50824163,  0.07430472,
       -0.17727741, -0.3407705 ,  0.56744999, -1.1545366 , -0.57175908,
        0.42881253,  0.41905238, -0.06575866, -0.33385471,  0.66821517,
       -1.74733627, -0.0485401 ,  0.15311076,  0.86423036, -0.14474725,
        2.65712533, -0.05447172, -0.53432362,  0.31597912,  0.40407797,
        0.22768482,  0.39576725, -0.29159359, -0.11262507, -0.13846773,
        0.17435764, -0.53750533,  0.94985298, -0.41448147, -0.10386168,
       -0.17551402, -0.22132863, -0.39945136,  0.21188116, -0.36097596])

## Keras implementation
 
Keras has an implementation of self-attention encapsulated in the `MultiHeadAttention` class. Before going to the attention module, the query, key value, goes through a dense layer. The output also goes through a dense layer (missing from Chollet's book, page 342). These three layers are initialized with Glorot's algorithm.

In [23]:
from tensorflow.keras.layers import MultiHeadAttention

att_layer = MultiHeadAttention(num_heads=1, 
                               key_dim=50, 
                               #kernel_initializer=tf.keras.initializers.ones(),
                               use_bias=False, 
                               attention_axes=(1,))

In [24]:
np.array([embeddings_seq]).shape

(1, 11, 50)

In [25]:
(attn_output, attn_scores) = att_layer(np.array([embeddings_seq]), 
          np.array([embeddings_seq]),
          np.array([embeddings_seq]),
         return_attention_scores=True)

2021-11-21 12:34:29.683919: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


The attention score for _ship:_

In [26]:
attn_scores[0][0][6]

<tf.Tensor: shape=(11,), dtype=float32, numpy=
array([0.0895291 , 0.09052993, 0.08993603, 0.09078447, 0.09066152,
       0.08840898, 0.09635282, 0.08957607, 0.09066152, 0.08840898,
       0.09515055], dtype=float32)>

The weight initial values with the 4 matrices

In [27]:
w_init = att_layer.weights
len(w_init)

4

In [28]:
w_init

[<tf.Variable 'multi_head_attention/query/kernel:0' shape=(50, 1, 50) dtype=float32, numpy=
 array([[[ 0.02252313,  0.0396619 ,  0.04085635, ..., -0.01897474,
           0.01869663, -0.022931  ]],
 
        [[ 0.01731944, -0.01797807, -0.03421662, ..., -0.01576941,
           0.00664544,  0.04849771]],
 
        [[ 0.01573787, -0.03923285,  0.01817504, ...,  0.01780319,
           0.0389211 ,  0.016796  ]],
 
        ...,
 
        [[ 0.03743019, -0.00185098,  0.03093588, ...,  0.01750521,
           0.00188751, -0.00760231]],
 
        [[-0.0391277 ,  0.04637659,  0.04846817, ...,  0.04243869,
          -0.03828867,  0.03432482]],
 
        [[-0.02236635, -0.04064306,  0.03302296, ...,  0.01464978,
           0.01181054, -0.0128864 ]]], dtype=float32)>,
 <tf.Variable 'multi_head_attention/key/kernel:0' shape=(50, 1, 50) dtype=float32, numpy=
 array([[[ 0.01114541, -0.03681173,  0.00774853, ..., -0.03556195,
          -0.02789038,  0.01469062]],
 
        [[ 0.01382273,  0.03663827, -0

### By-passing the dense layers

We create identity matrices to pass through the dense layers and recover the attention values and scores

In [29]:
i_50 = np.identity(50)

In [30]:
w_pt_50 = [i_50.reshape(50, 1, 50) for _ in range(3)] + [i_50.reshape(1, 50, 50)]

We set the new weights (pass through)

In [31]:
att_layer.set_weights(w_pt_50)

And we obtain the same results as the `self_attention()` function for _ship:_

In [32]:
att_layer(np.array([embeddings_seq]), 
          np.array([embeddings_seq]),
          np.array([embeddings_seq]),
         return_attention_scores=True)[1][0][0][6]

<tf.Tensor: shape=(11,), dtype=float32, numpy=
array([0.03030742, 0.03024589, 0.02764813, 0.04066233, 0.04593488,
       0.03426558, 0.55297625, 0.02968703, 0.04593488, 0.03426558,
       0.12807208], dtype=float32)>

In [33]:
att_layer(np.array([embeddings_seq]), 
          np.array([embeddings_seq]),
          np.array([embeddings_seq]),
         return_attention_scores=True)[0][0][6]

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([ 1.0387306 ,  0.10328995,  0.34260815, -0.43199396,  0.22365318,
       -0.09583741, -0.99261564,  0.6661626 ,  0.44239208, -0.79419196,
        0.56381553,  0.9921032 ,  0.02053553,  0.5082416 ,  0.07430474,
       -0.17727737, -0.34077048,  0.56745   , -1.1545366 , -0.5717591 ,
        0.42881253,  0.41905236, -0.06575862, -0.3338547 ,  0.6682152 ,
       -1.7473364 , -0.04854015,  0.15311077,  0.8642304 , -0.1447473 ,
        2.6571252 , -0.05447169, -0.5343237 ,  0.31597912,  0.40407795,
        0.22768481,  0.39576727, -0.29159355, -0.11262506, -0.13846776,
        0.17435764, -0.5375053 ,  0.9498529 , -0.41448146, -0.10386168,
       -0.17551401, -0.22132863, -0.3994514 ,  0.21188116, -0.36097592],
      dtype=float32)>

## Test with a simple matrix

Three words, dimension of embeddings: 4

In [34]:
test_input_sequence = np.array([[[1.0, 0.0, 0.0, 1.0],
                                 [0.0, 1.5, 1.0, 1.0],
                                 [0.0, 1.0, 1.0, 1.0]]])

In [35]:
test_input_sequence.shape

(1, 3, 4)

### Self-attention from the book

In [36]:
self_attention(test_input_sequence[0])

(array([[0.45186276, 0.68517155, 0.54813724, 1.        ],
        [0.10450673, 1.16085775, 0.89549327, 1.        ],
        [0.13872271, 1.10337221, 0.86127729, 1.        ]]),
 array([[0.45186276, 0.27406862, 0.27406862],
        [0.10450673, 0.53072895, 0.36476432],
        [0.13872271, 0.48418985, 0.37708743]]))

### Multihead attention from Keras

In [37]:
att_layer = MultiHeadAttention(num_heads=1, 
                               key_dim=4, 
                               #kernel_initializer=tf.keras.initializers.ones(), #my_init,
                               use_bias=False, attention_axes=(1,))

The multihead attention uses a Glorot initialization of the dense layers. The results will be different for those of `self_attention()`

In [38]:
att_layer(test_input_sequence, 
          test_input_sequence,
          test_input_sequence,
         return_attention_scores=True)

(<tf.Tensor: shape=(1, 3, 4), dtype=float32, numpy=
 array([[[ 0.37590456, -0.27779132, -0.36828738,  0.48588607],
         [ 0.35814926, -0.2999173 , -0.3627023 ,  0.48308557],
         [ 0.35688668, -0.30177456, -0.36227304,  0.48292154]]],
       dtype=float32)>,
 <tf.Tensor: shape=(1, 1, 3, 3), dtype=float32, numpy=
 array([[[[0.28883374, 0.35878244, 0.35238385],
          [0.34497535, 0.3100845 , 0.34494016],
          [0.3504005 , 0.3087359 , 0.3408636 ]]]], dtype=float32)>)

Weights of the dense layers

In [39]:
att_layer.weights #att_layer.get_weights()

[<tf.Variable 'multi_head_attention_1/query/kernel:0' shape=(4, 1, 4) dtype=float32, numpy=
 array([[[-0.32668424,  0.22653973, -0.2334781 , -0.2623816 ]],
 
        [[-0.08312231,  0.03224891,  0.4735502 ,  0.39178663]],
 
        [[ 0.291632  , -0.26512972, -0.15836379, -0.27192742]],
 
        [[ 0.2961046 ,  0.1269927 ,  0.24119568,  0.19220668]]],
       dtype=float32)>,
 <tf.Variable 'multi_head_attention_1/key/kernel:0' shape=(4, 1, 4) dtype=float32, numpy=
 array([[[-0.24973348, -0.44036877, -0.45794374,  0.38038564]],
 
        [[-0.46680564,  0.1920963 , -0.3139919 ,  0.11091363]],
 
        [[ 0.23258477,  0.5294845 , -0.1369949 ,  0.4493991 ]],
 
        [[ 0.39802974,  0.38902825,  0.23915654, -0.0436787 ]]],
       dtype=float32)>,
 <tf.Variable 'multi_head_attention_1/value/kernel:0' shape=(4, 1, 4) dtype=float32, numpy=
 array([[[ 0.4223336 ,  0.3806461 , -0.01654488, -0.2832061 ]],
 
        [[-0.14896375,  0.17501277,  0.24112153,  0.10876483]],
 
        [[-0.0788316

### By-passing the dense layers

We use weights of identity matrices

In [40]:
i_4 = np.identity(4)
w_pt_4 = [i_4.reshape(4, 1, 4) for _ in range(3)] + [i_4.reshape(1, 4, 4)]

In [41]:
w_pt_4

[array([[[1., 0., 0., 0.]],
 
        [[0., 1., 0., 0.]],
 
        [[0., 0., 1., 0.]],
 
        [[0., 0., 0., 1.]]]),
 array([[[1., 0., 0., 0.]],
 
        [[0., 1., 0., 0.]],
 
        [[0., 0., 1., 0.]],
 
        [[0., 0., 0., 1.]]]),
 array([[[1., 0., 0., 0.]],
 
        [[0., 1., 0., 0.]],
 
        [[0., 0., 1., 0.]],
 
        [[0., 0., 0., 1.]]]),
 array([[[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]]])]

We set these weights

In [42]:
att_layer.set_weights(w_pt_4)

Now we have the same results as with `self_attention()`

In [43]:
att_layer(test_input_sequence, 
          test_input_sequence,
          test_input_sequence,
         return_attention_scores=True)

(<tf.Tensor: shape=(1, 3, 4), dtype=float32, numpy=
 array([[[0.45186275, 0.6851716 , 0.54813725, 1.        ],
         [0.10450672, 1.1608577 , 0.89549327, 1.        ],
         [0.13872272, 1.1033722 , 0.86127734, 1.        ]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 1, 3, 3), dtype=float32, numpy=
 array([[[[0.45186275, 0.27406862, 0.27406862],
          [0.10450672, 0.53072894, 0.3647643 ],
          [0.13872272, 0.48418987, 0.37708744]]]], dtype=float32)>)